In [80]:
# Third-party library imports
import leafmap # leafmap==0.29.6 keplergl==0.3.2
import netCDF4 as nc
import numpy as np
import streamlit as st

time_index_wind = 6
level_index_wind = 2

netcdf_file = r"./data/netcdf/3D/reduced.nc"
netcdf_data = leafmap.read_netcdf(netcdf_file)

with nc.Dataset(netcdf_file, mode="r") as ds_reduced:
    max_u = np.nanmax(ds_reduced['u'][time_index_wind,level_index_wind,:,:])
    max_v = np.nanmax(ds_reduced['u'][time_index_wind,level_index_wind,:,:])
    max_velocity = (max_u**2+max_v**2)**0.5
    
    velocity_dict = {0.68: 0.06,
                         1.15: 0.05,
                         1.17: 0.03,
                         1.95: 0.025,
                         2.70: 0.016,
                         3.50: 0.0125,
                         }
    for key in velocity_dict:
        if max_velocity < key:
            velocity_scale = velocity_dict[key]
            break
        else:
            velocity_scale = 0.005
   
    test = (ds_reduced['lon'][128], ds_reduced['lat'][115])
    lon_lat = (test[0].item(), test[1].item())

m = leafmap.Map(zoom=17)
m.set_center(lon_lat[0], lon_lat[1])
m.add_basemap('Google Satellite')

# Add velocity
m.add_velocity(netcdf_data,
                zonal_speed='u',
                meridional_speed='v',
                latitude_dimension='lat',
                longitude_dimension='lon',
                level_dimension='lev',
                level_index=level_index_wind,
                time_index=time_index_wind,
                velocity_scale=velocity_scale,
                max_velocity=3,
                )

image_url = r"./images/base_simulation_alpha_075/N03/base_1800.png"
bounds_N03 = [[47.6588733206033766, 9.1718298413872255],
                    [47.6634905475628230, 9.1786643808656230]]
m.image_overlay(image_url, bounds_N03, name="2m air Temperature")
print(max_velocity)
m

2.1698451903619933


Map(center=[47.66094857351787, 9.17524696115189], controls=(ZoomControl(options=['position', 'zoom_in_text', '…